# Host Class Assignment of vOTUs using TensorFlow
This notebook uses a pariwise CRISPR infection network as well as host contig assignment to predict hosts of viruses with high accuracy

In [1]:
#Assignment of Host Based on Network Connections using Tensorflow
#Load modules
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import itertools
from tqdm import tqdm
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report

tf.get_logger().setLevel('ERROR')

In [2]:
#Load Inputs
data_file = 'bothnames_allClass_plusGC.txt'
col_names = ["vOTU_A", "vOTU_B", "Connections", "Pvalue", "Taxonomy2", "Taxonomy", "GC"]
df = pd.read_csv(data_file, names=col_names, header=None, sep='\t')
numclasses = df['Taxonomy'].nunique()

data_file2 = 'vOTU_number_Order.txt'
col_names2 = ["vOTU_A", "Count", "Taxonomy2"]
df2 = pd.read_csv(data_file2, names=col_names2, header=None, sep='\t')


In [3]:
# Parse data into usable form. Get one hot encoding of Taxonomy2

#Normalize Pvalue and GC difference to be within 0 and 1 and shift 0.01 to remove 0 values
df['Pvalue'] = (df['Pvalue'] - df['Pvalue'].min()) / (df['Pvalue'].max() - df['Pvalue'].min()) + 0.000001   
df['GC'] = (df['GC'] - df['GC'].min()) / (df['GC'].max() - df['GC'].min()) + 0.000001   

#One hot encode predicted taxonomy
one_hot = pd.get_dummies(df['Taxonomy2'])
df = df.drop('Taxonomy2',axis = 1)
df = df.join(one_hot)

#Weight taxonomy by p value
shape1 = df.shape[1]
df.update(df.iloc[:, 6:shape1].div(df.Pvalue, 0).div(1000000))

#Weight taxonomy by GC similarity
#one_hot = one_hot.add_suffix('_GC')
#df = df.join(one_hot)
#shape2 = df.shape[1]
#df.update(df.iloc[:, shape1:shape2].div(df.GC, 0).div(1000000))

#Weight by host taxonomy
one_hot = pd.get_dummies(df2['Taxonomy2'])
one_hot = one_hot.add_suffix('_contig')
df2 = df2.join(one_hot)
shape2 = df.shape[1]
df2['Count'] = df2['Count'].astype(int)
df2.update(df2.iloc[:, 2:shape2].mul(df2.Count, 0))  

#Drop uneeded parameters from dataset
df = df.drop('vOTU_B',axis = 1)
df = df.drop('Pvalue',axis = 1)
df = df.drop('GC',axis = 1)
df2 = df2.drop('Count',axis = 1)

#Sum weighted predicted taxonomies for each vOTU
dfsum = df.groupby(["vOTU_A", "Taxonomy"]).sum().reset_index()
dfsum2 = df2.groupby(["vOTU_A"]).sum().reset_index()


#Factorize taxonomy classes
dfsum["Taxonomy"], tax_key = pd.factorize(dfsum["Taxonomy"])

df_cd = pd.merge(dfsum, dfsum2, how='left', on = 'vOTU_A').fillna(0)
df_cd  = df_cd.drop('vOTU_A',axis = 1)
df_cd  = df_cd.drop('Connections',axis = 1)
#dfsum = dfsum.drop('vOTU_A',axis = 1)
#dfsum = dfsum.drop('Connections',axis = 1)
dfsum = df_cd
pd.set_option('display.max_columns', None)
#dfsum.head()
df_cd.head()


,Taxonomy,Acidithiobacillia,Acidobacteriia,Actinobacteria,Alphaproteobacteria,Anaerolineae,Aquificae,Bacilli,Bacteroidia,Betaproteobacteria,Blastocatellia,Caldilineae,Candidatus_Brocadiae,Chitinophagia,Chlamydiia,Chlorobia,Chloroflexia,Clostridia,Coprothermobacteria,Coriobacteriia,Cytophagia,Deinococci,Deltaproteobacteria,Dictyoglomia,Epsilonproteobacteria,Erysipelotrichi,Erysipelotrichia,Fibrobacteria,Flavobacteriia,Fusobacteriia,Gammaproteobacteria,Halobacteria,Hydrogenophilalia,Methanobacteria,Methanomicrobia,Negativicutes,Nitrospira,Opitutae,Phycisphaerae,Planctomycetia,Saprospiria,Sphingobacteriia,Spirochaetia,Synergistia,Thermococci,Thermodesulfobacteria,Thermomicrobia,Thermoplasmata,Thermoprotei,Thermotogae,Tissierellia,Verrucomicrobiae,Zetaproteobacteria,Acidimicrobiia_contig,Acidithiobacillia_contig,Acidobacteria_contig,Acidobacteriia_contig,Actinobacteria_contig,Actinomycetia_contig,Algae_contig,Alphaproteobacteria_contig,Anaerolineae_contig,Annelida_contig,Aquatic_contig,Aquificae_contig,Archaeoglobi_contig,Ardenticatenia_contig,Arthropoda:_contig,Artificial_contig,Bacilli_contig,Bacteroidia_contig,Balneolia_contig,Betaproteobacteria_contig,Bioreactor_contig,Bioremediation_contig,Biotransformation_contig,Birds_contig,Blastocatellia_contig,Built_contig,Caldilineae_contig,Caldisericia_contig,Caldithrixia_contig,Calditrichae_contig,Candidatus_contig,Caudoviricetes_contig,Chitinivibrionia_contig,Chitinophagia_contig,Chlamydiia_contig,Chlorobia_contig,Chloroflexia_contig,Chrysiogenetes_contig,Chthonomonadetes_contig,Clostridia_contig,Coprothermobacteria_contig,Coriobacteriia_contig,Cytophagia_contig,Deferribacteres_contig,Dehalococcoidia_contig,Deinococci_contig,Deltaproteobacteria_contig,Dictyoglomia_contig,Endomicrobia_contig,Epsilonproteobacteria_contig,Erysipelotrichia_contig,Fibrobacteria_contig,Fimbriimonadia_contig,Flavobacteriia_contig,Food_contig,Fungi_contig,Fusobacteriia_contig,Gammaproteobacteria_contig,Gemmatimonadetes_contig,Gloeobacteria_contig,Halobacteria_contig,Holophagae_contig,Huolimaviricetes_contig,Hydrogenophilalia_contig,Ignavibacteria_contig,Ktedonobacteria_contig,Lab_contig,Laserviricetes_contig,Lentisphaeria_contig,Limnochordia_contig,Mammals_contig,Mammals:_contig,Megaviricetes_contig,Melainabacteria_contig,Methanobacteria_contig,Methanococci_contig,Methanomicrobia_contig,Microbial_contig,Modeled_contig,Mollicutes_contig,Mollusca_contig,Negativicutes_contig,Nitrososphaeria_contig,Nitrospinia_contig,Nitrospira_contig,Oligoflexia_contig,Opitutae_contig,Phycisphaerae_contig,Planctomycetia_contig,Plants_contig,Porifera_contig,Rhodothermia_contig,Rubrobacteria_contig,Saprospiria_contig,Solibacteres_contig,Solid_contig,Spartobacteria_contig,Sphingobacteriia_contig,Spirochaetia_contig,Synergistia_contig,Tectiliviricetes_contig,Terrestrial_contig,Thermoanaerobaculia_contig,Thermobacula_contig,Thermococci_contig,Thermodesulfobacteria_contig,Thermodesulfovibrionia_contig,Thermoflexia_contig,Thermoleophilia_contig,Thermomicrobia_contig,Thermoplasmata_contig,Thermoprotei_contig,Thermotogae_contig,Tissierellia_contig,Unclassified_contig,Verrucomicrobiae_contig,Vicinamibacteria_contig,WWTP_contig,Wastewater_contig,Zetaproteobacteria_contig,unclassified_contig
0,0,0.0,0.0,329.254345,0.000000,0.0,0.0,0.000063,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.017205,0.0,0.066825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,310.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0

In [4]:
#Create training and test set
norm_train_X, norm_test_X = train_test_split(dfsum, test_size = 0.2)
#norm_train_X.head()
train_Y = norm_train_X.pop("Taxonomy")
test_Y = norm_test_X.pop("Taxonomy")

dfsum_X = dfsum
dfsum_Y = dfsum_X.pop("Taxonomy")

#y = dfsum["Taxonomy"]
#x = dfsum.drop("Taxonomy", axis = 1)

#Very unbalanced set is difficult to stratify. Some taxonomic class will drop, and reduce test accuracy no matter how you split it. 
#norm_train_X, norm_test_X, train_Y, test_Y = train_test_split(x, y,
#                                                    stratify=y, 
#                                                    test_size=0.1)

In [19]:
def base_model():
    inputs = tf.keras.layers.Input(shape=(len(norm_train_X.columns)))

    x = tf.keras.layers.Dense(102, activation='relu')(inputs)
    x = tf.keras.layers.Dense(51, activation='relu')(x)
    #x = tf.keras.layers.Dropout(0.2)(x)
    #x = tf.keras.layers.Dropout(0.15)(x)
    outputs = tf.keras.layers.Dense(numclasses, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer = 'adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    return model

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=150)
model = base_model()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 173)]             0         
                                                                 
 dense_4 (Dense)             (None, 102)               17748     
                                                                 
 dense_5 (Dense)             (None, 51)                5253      
                                                                 
 dense_6 (Dense)             (None, 51)                2652      
                                                                 
Total params: 25,653
Trainable params: 25,653
Non-trainable params: 0
_________________________________________________________________


In [21]:
#norm_train_X.head()
history = model.fit(norm_train_X.values, train_Y,
                    epochs=400, batch_size=307,
                    validation_data=(norm_test_X.values, test_Y), callbacks=[callback])

Epoch 1/400
75/75 [==============================] - 0s 3ms/step - loss: 0.0941 - accuracy: 0.9656 - val_loss: 0.0951 - val_accuracy: 0.9641
Epoch 2/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0922 - accuracy: 0.9669 - val_loss: 0.0977 - val_accuracy: 0.9650
Epoch 3/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0883 - accuracy: 0.9672 - val_loss: 0.1030 - val_accuracy: 0.9625
Epoch 4/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0970 - accuracy: 0.9673 - val_loss: 0.1083 - val_accuracy: 0.9645
Epoch 5/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0968 - accuracy: 0.9670 - val_loss: 0.1109 - val_accuracy: 0.9624
Epoch 6/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0925 - accuracy: 0.9679 - val_loss: 0.1087 - val_accuracy: 0.9639
Epoch 7/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0880 - accuracy: 0.9684 - val_loss: 0.1105 - val_accuracy: 0.9643
Epoch 8/400
7

Epoch 59/400
75/75 [==============================] - 0s 3ms/step - loss: 0.0838 - accuracy: 0.9697 - val_loss: 0.1709 - val_accuracy: 0.9566
Epoch 60/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0824 - accuracy: 0.9696 - val_loss: 0.1683 - val_accuracy: 0.9557
Epoch 61/400
75/75 [==============================] - 0s 2ms/step - loss: 0.0818 - accuracy: 0.9696 - val_loss: 0.1743 - val_accuracy: 0.9555
Epoch 62/400
75/75 [==============================] - 0s 3ms/step - loss: 0.0817 - accuracy: 0.9686 - val_loss: 0.1830 - val_accuracy: 0.9552
Epoch 63/400
75/75 [==============================] - 0s 3ms/step - loss: 0.0848 - accuracy: 0.9696 - val_loss: 0.1728 - val_accuracy: 0.9555
Epoch 64/400
75/75 [==============================] - 0s 3ms/step - loss: 0.0828 - accuracy: 0.9697 - val_loss: 0.1845 - val_accuracy: 0.9552
Epoch 65/400
75/75 [==============================] - 0s 3ms/step - loss: 0.0854 - accuracy: 0.9688 - val_loss: 0.1764 - val_accuracy: 0.9553
Epoch 

In [23]:
#Get Class Specific metrics
predictions = np.argmax(model.predict(dfsum_X), axis=-1)

clsf_report = pd.DataFrame(classification_report(y_true = predictions, y_pred = dfsum_Y, output_dict=True)).transpose()
clsf_report.drop(clsf_report.tail(3).index,
        inplace = True)


clsf_report.insert(0, 'Names', list(tax_key.values))
clsf_report




/Users/BFremin/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/BFremin/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/BFremin/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Names,precision,recall,f1-score,support
0,Actinobacteria,0.986439,0.965857,0.976040,2636.0
1,Bacilli,0.952617,0.970811,0.961628,2775.0
2,Clostridia,0.988765,0.976733,0.982712,10272.0
3,Fusobacteriia,0.992314,0.994863,0.993587,1168.0
4,Gammaproteobacteria,0.961487,0.980813,0.971054,2189.0
5,Bacteroidia,0.991118,0.992832,0.991974,4046.0
6,Betaproteobacteria,0.939636,0.910500,0.924839,1419.0
7,Chlamydiia,0.555556,0.729167,0.630631,48.0
8,Thermodesulfobacteria,0.882353,1.000000,0.937500,15.0
9,Verrucomicrobiae,0.988636,0.977528,0.983051,89.0


In [45]:
#Make predictions on vOTUs where both hosts are unknown

#Load Inputs
data_file3 = 'filtered_allClass_vOTUs.txt'
col_names3 = ["vOTU_A", "vOTU_B", "Connections", "Pvalue", "Taxonomy2"]
df_new = pd.read_csv(data_file3, names=col_names3, header=None, sep='\t')
numclasses_new = df_new['Taxonomy2'].nunique()
df_new.head()

,vOTU_A,vOTU_B,Connections,Pvalue,Taxonomy2
0,Sg_000787,Sg_004751,171,2.009161e-44,Actinobacteria
1,Sg_000787,Sg_005966,281,2.481592e-54,Actinobacteria
2,Sg_000787,Sg_006191,293,1.598252e-47,Actinobacteria
3,Sg_000787,Sg_006570,138,1.084720e-35,Actinobacteria
4,Sg_000787,Sg_006768,265,7.802467e-45,Actinobacteria


In [46]:
#Get data in appropriate format
# Parse data into usable form. Get one hot encoding of Taxonomy2

#Normalize Pvalue and GC difference to be within 0 and 1 and shift 0.01 to remove 0 values
df_new['Pvalue'] = (df_new['Pvalue'] - df_new['Pvalue'].min()) / (df_new['Pvalue'].max() - df_new['Pvalue'].min()) + 0.000001   

#df_new = df_new[~df_new['Taxonomy2'].isin(list(tax_key))]

#One hot encode predicted taxonomy
one_hot = pd.get_dummies(df_new['Taxonomy2'])
df_new = df_new.drop('Taxonomy2',axis = 1)
df_new = df_new.join(one_hot)

#Weight taxonomy by p value
shape1 = df_new.shape[1]
df_new.update(df_new.iloc[:, 6:shape1].div(df_new.Pvalue, 0).div(1000000))

#Weight taxonomy by GC similarity
#one_hot = one_hot.add_suffix('_GC')
#df = df.join(one_hot)
#shape2 = df.shape[1]
#df.update(df.iloc[:, shape1:shape2].div(df.GC, 0).div(1000000))                  

#Drop uneeded parameters from dataset
df_new = df_new.drop('vOTU_B',axis = 1)
df_new = df_new.drop('Pvalue',axis = 1)

#Sum weighted predicted taxonomies for each vOTU
dfsum_new = df_new.groupby(["vOTU_A"]).sum().reset_index()

dfsum_new = dfsum_new.drop('Connections',axis = 1)

df_cd2 = pd.merge(dfsum_new, dfsum2, how='outer', on = 'vOTU_A').fillna(0)
dfsum_new = df_cd2
pd.set_option('display.max_columns', None)
dfsum_new.head()


,vOTU_A,Acidithiobacillia,Acidobacteriia,Actinobacteria,Alphaproteobacteria,Anaerolineae,Aquificae,Bacilli,Bacteroidia,Betaproteobacteria,Blastocatellia,Caldilineae,Candidatus_Brocadiae,Chitinophagia,Chlamydiia,Chlorobia,Chloroflexia,Clostridia,Coprothermobacteria,Coriobacteriia,Cytophagia,Deinococci,Deltaproteobacteria,Dictyoglomia,Epsilonproteobacteria,Erysipelotrichi,Erysipelotrichia,Fibrobacteria,Flavobacteriia,Fusobacteriia,Gammaproteobacteria,Halobacteria,Hydrogenophilalia,Methanobacteria,Methanomicrobia,Negativicutes,Nitrospira,Opitutae,Phycisphaerae,Planctomycetia,Saprospiria,Sphingobacteriia,Spirochaetia,Synergistia,Thermococci,Thermodesulfobacteria,Thermomicrobia,Thermoplasmata,Thermoprotei,Thermotogae,Tissierellia,Verrucomicrobiae,Zetaproteobacteria,Acidimicrobiia_contig,Acidithiobacillia_contig,Acidobacteria_contig,Acidobacteriia_contig,Actinobacteria_contig,Actinomycetia_contig,Algae_contig,Alphaproteobacteria_contig,Anaerolineae_contig,Annelida_contig,Aquatic_contig,Aquificae_contig,Archaeoglobi_contig,Ardenticatenia_contig,Arthropoda:_contig,Artificial_contig,Bacilli_contig,Bacteroidia_contig,Balneolia_contig,Betaproteobacteria_contig,Bioreactor_contig,Bioremediation_contig,Biotransformation_contig,Birds_contig,Blastocatellia_contig,Built_contig,Caldilineae_contig,Caldisericia_contig,Caldithrixia_contig,Calditrichae_contig,Candidatus_contig,Caudoviricetes_contig,Chitinivibrionia_contig,Chitinophagia_contig,Chlamydiia_contig,Chlorobia_contig,Chloroflexia_contig,Chrysiogenetes_contig,Chthonomonadetes_contig,Clostridia_contig,Coprothermobacteria_contig,Coriobacteriia_contig,Cytophagia_contig,Deferribacteres_contig,Dehalococcoidia_contig,Deinococci_contig,Deltaproteobacteria_contig,Dictyoglomia_contig,Endomicrobia_contig,Epsilonproteobacteria_contig,Erysipelotrichia_contig,Fibrobacteria_contig,Fimbriimonadia_contig,Flavobacteriia_contig,Food_contig,Fungi_contig,Fusobacteriia_contig,Gammaproteobacteria_contig,Gemmatimonadetes_contig,Gloeobacteria_contig,Halobacteria_contig,Holophagae_contig,Huolimaviricetes_contig,Hydrogenophilalia_contig,Ignavibacteria_contig,Ktedonobacteria_contig,Lab_contig,Laserviricetes_contig,Lentisphaeria_contig,Limnochordia_contig,Mammals_contig,Mammals:_contig,Megaviricetes_contig,Melainabacteria_contig,Methanobacteria_contig,Methanococci_contig,Methanomicrobia_contig,Microbial_contig,Modeled_contig,Mollicutes_contig,Mollusca_contig,Negativicutes_contig,Nitrososphaeria_contig,Nitrospinia_contig,Nitrospira_contig,Oligoflexia_contig,Opitutae_contig,Phycisphaerae_contig,Planctomycetia_contig,Plants_contig,Porifera_contig,Rhodothermia_contig,Rubrobacteria_contig,Saprospiria_contig,Solibacteres_contig,Solid_contig,Spartobacteria_contig,Sphingobacteriia_contig,Spirochaetia_contig,Synergistia_contig,Tectiliviricetes_contig,Terrestrial_contig,Thermoanaerobaculia_contig,Thermobacula_contig,Thermococci_contig,Thermodesulfobacteria_contig,Thermodesulfovibrionia_contig,Thermoflexia_contig,Thermoleophilia_contig,Thermomicrobia_contig,Thermoplasmata_contig,Thermoprotei_contig,Thermotogae_contig,Tissierellia_contig,Unclassified_contig,Verrucomicrobiae_contig,Vicinamibacteria_contig,WWTP_contig,Wastewater_contig,Zetaproteobacteria_contig,unclassified_contig
0,Sg_000787,0.0,0.0,715.96545,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,310.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sg_001325,0.0,0.0,0.00000,0.0,0.0,0

In [47]:
#Predict host
dfsum_new_X = dfsum_new
dfsum_new_Y = dfsum_new_X.pop("vOTU_A")

pred = model.predict(dfsum_new_X)

In [48]:
print(pred, dfsum_new_Y[np.argmax(pred)])

[[9.9999905e-01 6.9202730e-07 2.8819411e-13 ... 2.6379746e-38
  0.0000000e+00 0.0000000e+00]
 [2.4505276e-20 9.9984443e-01 1.4505188e-04 ... 0.0000000e+00
  0.0000000e+00 9.5299196e-32]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [1.9680317e-03 8.2972972e-03 1.7916270e-02 ... 5.4104152e-07
  6.6282078e-18 1.9648352e-16]
 [5.6106718e-03 2.9680233e-02 1.0190655e-01 ... 4.3577003e-15
  0.0000000e+00 2.1175193e-31]
 [2.7538708e-03 1.1848912e-02 1.9696245e-02 ... 1.3809975e-07
  2.6130752e-19 6.4862812e-17]] Sg_014546


In [49]:
outfile=open('classlabel.txt','w')
for i in range(len(dfsum_new_Y)):  
    outfile.write(str(dfsum_new_Y[i]) + "\t" + str(tax_key[np.argmax(pred[i])]) + "\t" + str(max(pred[i])))
    outfile.write('\n')
    print(dfsum_new_Y[i], tax_key[np.argmax(pred[i])], max(pred[i]))
outfile.close()
    

Sg_000787 Actinobacteria 0.99999905
Sg_001325 Bacilli 0.99984443
Sg_002428 Clostridia 1.0
Sg_002918 Fusobacteriia 1.0
Sg_002925 Fusobacteriia 1.0
Sg_003003 Gammaproteobacteria 1.0
Sg_003109 Gammaproteobacteria 1.0
Sg_004179 Clostridia 0.9996203
Sg_004398 Bacteroidia 1.0
Sg_004631 Bacteroidia 1.0
Sg_004751 Actinobacteria 1.0
Sg_004769 Bacteroidia 0.99999976
Sg_004819 Bacteroidia 1.0
Sg_004876 Bacteroidia 1.0
Sg_004883 Bacteroidia 1.0
Sg_004899 Bacteroidia 0.9999999
Sg_004945 Bacilli 0.9604924
Sg_004955 Clostridia 1.0
Sg_004959 Bacteroidia 1.0
Sg_005020 Bacteroidia 1.0
Sg_005189 Bacteroidia 1.0
Sg_005190 Bacteroidia 1.0
Sg_005395 Bacteroidia 0.9999999
Sg_005414 Clostridia 1.0
Sg_005463 Clostridia 1.0
Sg_005646 Clostridia 1.0
Sg_005652 Bacteroidia 0.99999475
Sg_005966 Actinobacteria 0.99999845
Sg_006069 Betaproteobacteria 0.9999833
Sg_006090 Bacteroidia 1.0
Sg_006191 Actinobacteria 1.0
Sg_006240 Clostridia 1.0
Sg_006570 Actinobacteria 1.0
Sg_006590 Clostridia 0.9700769
Sg_006710 Clostridi

Sg_043021 Clostridia 1.0
Sg_043063 Clostridia 1.0
Sg_043094 Clostridia 1.0
Sg_043107 Clostridia 1.0
Sg_043109 Clostridia 1.0
Sg_043114 Clostridia 1.0
Sg_043139 Bacteroidia 1.0
Sg_043167 Clostridia 1.0
Sg_043232 Clostridia 1.0
Sg_043237 Clostridia 1.0
Sg_043241 Clostridia 1.0
Sg_043248 Clostridia 1.0
Sg_043293 Clostridia 1.0
Sg_043300 Clostridia 1.0
Sg_043309 Clostridia 1.0
Sg_043376 Actinobacteria 1.0
Sg_043381 Clostridia 1.0
Sg_043382 Gammaproteobacteria 0.9999733
Sg_043395 Actinobacteria 1.0
Sg_043408 Clostridia 1.0
Sg_043427 Clostridia 1.0
Sg_043436 Clostridia 1.0
Sg_043439 Clostridia 1.0
Sg_043444 Clostridia 1.0
Sg_043452 Clostridia 1.0
Sg_043524 Actinobacteria 0.9732861
Sg_043526 Clostridia 0.9999999
Sg_043533 Clostridia 1.0
Sg_043549 Clostridia 1.0
Sg_043559 Bacilli 0.9999999
Sg_043562 Bacteroidia 0.99975246
Sg_043568 Clostridia 1.0
Sg_043580 Clostridia 1.0
Sg_043584 Clostridia 1.0
Sg_043601 Clostridia 1.0
Sg_043608 Clostridia 0.8600301
Sg_043637 Clostridia 1.0
Sg_043694 Fusobact

Sg_050263 Clostridia 1.0
Sg_050302 Clostridia 1.0
Sg_050391 Clostridia 1.0
Sg_050409 Bacteroidia 1.0
Sg_050413 Clostridia 1.0
Sg_050455 Clostridia 1.0
Sg_050464 Clostridia 1.0
Sg_050475 Clostridia 1.0
Sg_050484 Clostridia 1.0
Sg_050519 Clostridia 0.99989617
Sg_050531 Clostridia 1.0
Sg_050550 Gammaproteobacteria 1.0
Sg_050583 Bacilli 0.9999901
Sg_050616 Clostridia 1.0
Sg_050620 Clostridia 1.0
Sg_050633 Clostridia 1.0
Sg_050678 Bacilli 1.0
Sg_050696 Clostridia 1.0
Sg_050722 Clostridia 1.0
Sg_050735 Bacteroidia 1.0
Sg_050739 Clostridia 1.0
Sg_050742 Actinobacteria 0.98075
Sg_050746 Clostridia 1.0
Sg_050759 Clostridia 0.99999976
Sg_050770 Clostridia 1.0
Sg_050803 Clostridia 0.99444455
Sg_050837 Thermoprotei 1.0
Sg_050838 Clostridia 0.99999976
Sg_050854 Clostridia 1.0
Sg_050868 Clostridia 1.0
Sg_050902 Clostridia 1.0
Sg_050934 Clostridia 1.0
Sg_050940 Clostridia 1.0
Sg_050984 Clostridia 1.0
Sg_050985 Clostridia 1.0
Sg_051000 Clostridia 1.0
Sg_051046 Clostridia 1.0
Sg_051052 Clostridia 1.0
S

Sg_066130 Clostridia 1.0
Sg_066182 Clostridia 1.0
Sg_066204 Clostridia 0.9996886
Sg_066212 Clostridia 0.9999876
Sg_066259 Bacteroidia 0.99430496
Sg_066289 Bacteroidia 0.99999976
Sg_066452 Betaproteobacteria 0.9974855
Sg_066458 Clostridia 0.9999943
Sg_066484 Clostridia 1.0
Sg_066485 Gammaproteobacteria 0.99999785
Sg_066494 Clostridia 1.0
Sg_066496 Negativicutes 0.99853814
Sg_066500 Clostridia 0.999979
Sg_066504 Clostridia 1.0
Sg_066523 Gammaproteobacteria 1.0
Sg_066586 Clostridia 0.99999595
Sg_066691 Gammaproteobacteria 1.0
Sg_066707 Clostridia 0.99953496
Sg_066710 Bacteroidia 1.0
Sg_066765 Actinobacteria 1.0
Sg_066886 Fusobacteriia 1.0
Sg_066941 Actinobacteria 0.9044643
Sg_066973 Clostridia 0.99999857
Sg_067054 Bacteroidia 0.9999999
Sg_067127 Clostridia 1.0
Sg_067128 Gammaproteobacteria 1.0
Sg_067191 Clostridia 1.0
Sg_067201 Erysipelotrichia 1.0
Sg_067206 Bacteroidia 1.0
Sg_067224 Clostridia 0.9999608
Sg_067253 Clostridia 1.0
Sg_067277 Actinobacteria 1.0
Sg_067287 Erysipelotrichia 1.0


Sg_080638 Clostridia 1.0
Sg_080652 Bacteroidia 0.9999889
Sg_080653 Clostridia 1.0
Sg_080654 Bacteroidia 1.0
Sg_080667 Clostridia 0.9997203
Sg_080673 Clostridia 1.0
Sg_080696 Bacteroidia 1.0
Sg_080702 Clostridia 1.0
Sg_080708 Clostridia 1.0
Sg_080725 Clostridia 1.0
Sg_080775 Clostridia 0.9999876
Sg_080783 Clostridia 1.0
Sg_080793 Clostridia 1.0
Sg_080799 Betaproteobacteria 0.99963987
Sg_080802 Bacteroidia 1.0
Sg_080824 Clostridia 1.0
Sg_080859 Betaproteobacteria 0.9915399
Sg_080867 Negativicutes 0.9998702
Sg_080881 Clostridia 1.0
Sg_080890 Bacteroidia 1.0
Sg_080910 Bacteroidia 1.0
Sg_080912 Gammaproteobacteria 1.0
Sg_080914 Bacilli 0.9999993
Sg_080924 Clostridia 1.0
Sg_080937 Bacteroidia 1.0
Sg_080956 Clostridia 1.0
Sg_080966 Clostridia 1.0
Sg_081022 Gammaproteobacteria 1.0
Sg_081023 Negativicutes 0.99983656
Sg_081070 Clostridia 1.0
Sg_081071 Clostridia 1.0
Sg_081087 Negativicutes 0.99999976
Sg_081113 Bacteroidia 0.99999905
Sg_081115 Bacteroidia 1.0
Sg_081129 Bacteroidia 0.9999682
Sg_08

...

vOTU_270062 Betaproteobacteria 0.5424717
vOTU_270108 Nitrospira 0.99999964
vOTU_270246 Betaproteobacteria 0.57042825
vOTU_270292 Bacteroidia 0.67644143
vOTU_270380 Methanomicrobia 1.0
vOTU_270423 Erysipelotrichia 0.6968237
vOTU_270495 Bacteroidia 0.99848133
vOTU_270509 Alphaproteobacteria 0.56258154
vOTU_271059 Bacteroidia 0.998355
vOTU_271135 Thermoprotei 0.99999857
vOTU_271452 Bacteroidia 0.98480505
vOTU_271569 Gammaproteobacteria 0.98421854
vOTU_271830 Clostridia 0.4601756
vOTU_271992 Gammaproteobacteria 0.56589
vOTU_272186 Negativicutes 0.54876506
vOTU_272222 Clostridia 0.94755083
vOTU_272382 Gammaproteobacteria 0.5573292
vOTU_272411 Gammaproteobacteria 0.28950813
vOTU_272607 Clostridia 0.93503284
vOTU_272675 Anaerolineae 0.99999976
vOTU_272928 Alphaproteobacteria 0.58325106
vOTU_273301 Bacilli 0.8004352
vOTU_273303 Flavobacteriia 0.9983725
vOTU_273314 Betaproteobacteria 0.57042825
vOTU_273407 Alphaproteobacteria 0.7267822
vOTU_273430 Actinobacteria 0.9965581
vOTU_273636 Bacteroidi

vOTU_360315 Actinobacteria 0.9431275
vOTU_360522 Clostridia 0.94065994
vOTU_360625 Deltaproteobacteria 0.99943966
vOTU_360725 Clostridia 0.965837
vOTU_360855 Bacteroidia 1.0
vOTU_360861 Dictyoglomia 0.9998832
vOTU_360865 Bacilli 0.5618881
vOTU_361084 Actinobacteria 0.84203064
vOTU_361096 Gammaproteobacteria 0.4948219
vOTU_361165 Actinobacteria 0.9363505
vOTU_361318 Gammaproteobacteria 0.43759742
vOTU_361346 Clostridia 0.95298356
vOTU_361429 Gammaproteobacteria 0.969936
vOTU_361499 Betaproteobacteria 0.62495244
vOTU_361672 Clostridia 0.9017545
vOTU_361972 Actinobacteria 0.8810398
vOTU_362026 Bacilli 0.9976896
vOTU_362740 Coriobacteriia 0.99356294
vOTU_362843 Planctomycetia 1.0
vOTU_362871 Deltaproteobacteria 1.0
vOTU_363042 Erysipelotrichia 0.8333278
vOTU_363162 Clostridia 0.96665615
vOTU_363248 Thermoprotei 0.9999826
vOTU_363254 Alphaproteobacteria 0.7185515
vOTU_363556 Clostridia 0.99997544
vOTU_363721 Clostridia 0.986019
vOTU_364135 Halobacteria 1.0
vOTU_364152 Gammaproteobacteria 0.

vOTU_472066 Actinobacteria 0.9999609
vOTU_472689 Betaproteobacteria 0.811548
vOTU_472847 Betaproteobacteria 0.57042825
vOTU_473025 Clostridia 0.961698
vOTU_473997 Clostridia 0.94515115
vOTU_474210 Alphaproteobacteria 0.4511021
vOTU_474875 Clostridia 0.95881814
vOTU_475166 Clostridia 0.96112263
vOTU_475323 Clostridia 0.94510686
vOTU_475715 Gammaproteobacteria 0.58554316
vOTU_475879 Bacteroidia 0.8957427
vOTU_476135 Gammaproteobacteria 0.37460816
vOTU_476610 Gammaproteobacteria 0.89069515
vOTU_476819 Clostridia 0.86809856
vOTU_477088 Alphaproteobacteria 0.9998791
vOTU_477308 Flavobacteriia 0.8346276
vOTU_477394 Clostridia 0.9646886
vOTU_477449 Bacteroidia 0.35002676
vOTU_477461 Clostridia 0.9373999
vOTU_477776 Betaproteobacteria 0.9682483
vOTU_478163 Erysipelotrichia 0.99906594
vOTU_478617 Betaproteobacteria 0.7619899
vOTU_478741 Gammaproteobacteria 0.969936
vOTU_479036 Clostridia 0.8363106
vOTU_479597 Clostridia 0.5181645
vOTU_480145 Bacteroidia 0.47555804
vOTU_480255 Gammaproteobacteri

vOTU_626517 Bacteroidia 0.8957427
vOTU_626878 Bacilli 0.7440902
vOTU_627044 Bacteroidia 1.0
vOTU_627308 Flavobacteriia 0.9125208
vOTU_627353 Clostridia 0.86809856
vOTU_627675 Betaproteobacteria 0.8866588
vOTU_627850 Clostridia 0.9739402
vOTU_628170 Thermoprotei 0.999998
vOTU_628567 Bacteroidia 0.98480505
vOTU_629477 Bacteroidia 0.9999461
vOTU_629976 Betaproteobacteria 0.7122696
vOTU_630509 Betaproteobacteria 0.4386104
vOTU_630920 Gammaproteobacteria 0.31626803
vOTU_631118 Betaproteobacteria 0.6682553
vOTU_631165 Betaproteobacteria 0.86839086
vOTU_631445 Betaproteobacteria 0.85579294
vOTU_631679 Gammaproteobacteria 0.37460816
vOTU_632962 Clostridia 0.9660055
vOTU_633270 Alphaproteobacteria 0.7076066
vOTU_633404 Alphaproteobacteria 0.46323937
vOTU_633869 Betaproteobacteria 0.58032715
vOTU_633928 Gammaproteobacteria 0.89069515
vOTU_634007 Alphaproteobacteria 0.8728434
vOTU_634325 Bacteroidia 0.8857659
vOTU_635433 Negativicutes 0.96725667
vOTU_635536 Gammaproteobacteria 0.969936
vOTU_63615

vOTU_872237 Clostridia 0.9848905
vOTU_872290 Gammaproteobacteria 0.7309322
vOTU_872890 Clostridia 0.9429693
vOTU_873167 Clostridia 0.94337493
vOTU_873187 Clostridia 0.96558005
vOTU_873383 Bacteroidia 0.9997998
vOTU_873453 Bacteroidia 0.9783618
vOTU_874084 Clostridia 0.968204
vOTU_874085 Clostridia 0.9735788
vOTU_874240 Clostridia 0.9350678
vOTU_874582 Clostridia 0.9628973
vOTU_874625 Methanobacteria 0.8905373
vOTU_875257 Clostridia 0.9593987
vOTU_875426 Clostridia 0.96022266
vOTU_875991 Gammaproteobacteria 0.79649514
vOTU_876034 Clostridia 0.85760134
vOTU_876265 Clostridia 0.9239674
vOTU_876352 Clostridia 0.99530834
vOTU_877173 Bacilli 0.996774
vOTU_877703 Bacteroidia 1.0
vOTU_878126 Negativicutes 0.82394576
vOTU_879230 Gammaproteobacteria 0.969936
vOTU_879232 Clostridia 0.98043066
vOTU_879372 Verrucomicrobiae 0.9994672
vOTU_879668 Negativicutes 0.65948534
vOTU_881524 Chlorobia 0.96448463
vOTU_881525 Clostridia 0.96583533
vOTU_881631 Clostridia 0.95659095
vOTU_883284 Clostridia 0.93189

In [24]:
model.save('saved_model/assignclass_model')

2023-02-13 14:25:15.370215: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
